In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import math
import random
import scipy

# Load data from Excel file
# data = pd.read_excel('C:/Users/Dear User/Neural Networks/Excercise2/Datasets/classification-iris.xlsx', header=None).values
data = pd.read_excel('C:/Users/Dear User/Neural Networks/Excercise2/Datasets/dataset_diabetic.xlsx', header=None).values
# data = pd.read_excel('C:/Users/Dear User/Neural Networks/Excercise2/Datasets/classification-glass.xlsx', header=None).values

# data = pd.read_excel('dataset_diabetic.xlsx', header=None).values  
# data = pd.read_excel('classification-glass.xlsx', header=None).values  

# Shuffle the data
data = shuffle(data, random_state=42)

num_data = data.shape[0]
num_col = data.shape[1]
num_class = len(np.unique(data[:,num_col-1]))

# Normalize the input data
for ii in range(num_col-1): 
    data[:, ii] = (data[:, ii] / np.max(data[:, ii]) )
    
# Split the dataset into a training and testing set
# X_train, X_test,y_train, y_test = train_test_split(data[:num_data, :3], data[:num_data,3], test_size=0.25, random_state=42)

split_ratio_train = 0.7

split_line_number = int(np.shape(data)[0] * split_ratio_train)
X_train = data[:split_line_number, :(num_col-1)]
y_train = data[:split_line_number, num_col-1]
num_train = len(X_train)

X_test = data[split_line_number:len(data), :(num_col-1)]
y_test = data[split_line_number:len(data), num_col-1] 
num_test = len(X_test)

# Define the number of input, hidden, and output neurons
input_neurons = X_train.shape[1]
# for iris
l1_neurons = 10
l2_neurons = 20

# for diabet
# l1_neurons = 10
# l2_neurons = 15
output_neurons = num_class

# Convert labels to one-hot encoding (necessary for multi-class classification)
y_one_hot_train = np.zeros((num_train, num_class))
y_one_hot_train[np.arange(num_train), data[:num_train, num_col-1].astype(int)-1] = 1

y_one_hot_test = np.zeros((num_test, num_class))
y_one_hot_test[np.arange(num_test), data[num_train:, num_col-1].astype(int)-1] = 1


# Initialize the weights with random values in range (-1,1)
np.random.seed(1)
w1 = 2 * np.random.random((input_neurons, l1_neurons)) - 1
w2 = 2 * np.random.random((l1_neurons, l2_neurons)) - 1
w3 = 2 * np.random.random((l2_neurons, output_neurons)) - 1
# print('w1: ', w1, 'w2: ',w2, 'w3: ',w3)

# Initialize momentum parameters 
vw1 = np.zeros((input_neurons, l1_neurons))
vw2 = np.zeros((l1_neurons, l2_neurons))
vw3 = np.zeros((l2_neurons, output_neurons))

v𝛼1,v𝛽1,v𝛾1,v𝛿1 = np.zeros((l1_neurons, 1)) ,np.zeros((l1_neurons, 1)) ,np.zeros((l1_neurons, 1)) ,np.zeros((l1_neurons, 1))
v𝛼2,v𝛽2,v𝛾2,v𝛿2 = np.zeros((l2_neurons,1)),np.zeros((l2_neurons,1)),np.zeros((l2_neurons,1)),np.zeros((l2_neurons,1))
v𝛼3,v𝛽3,v𝛾3,v𝛿3 = np.zeros((output_neurons,1)),np.zeros((output_neurons,1)),np.zeros((output_neurons,1)),np.zeros((output_neurons,1))

v𝜃11,v𝜃12,v𝜃13,v𝜃14 = np.zeros((l1_neurons, 1)) ,np.zeros((l1_neurons, 1)),np.zeros((l1_neurons, 1)),np.zeros((l1_neurons, 1))
v𝜃21,v𝜃22,v𝜃23,v𝜃24 = np.zeros((l2_neurons,1)),np.zeros((l2_neurons,1)),np.zeros((l2_neurons,1)),np.zeros((l2_neurons,1))
v𝜃31,v𝜃32,v𝜃33,v𝜃34 = np.zeros((output_neurons,1)),np.zeros((output_neurons,1)),np.zeros((output_neurons,1)),np.zeros((output_neurons,1))
beta = 0.9

#𝜃ij => i: layer j:function_num
𝜃11,𝜃12,𝜃13,𝜃14 =  np.random.uniform(-1, 1, (l1_neurons, 1)) ,np.random.uniform(-1, 1, (l1_neurons, 1)),np.random.uniform(-1, 1, (l1_neurons, 1)),np.random.uniform(-1, 1, (l1_neurons, 1))
𝜃21,𝜃22,𝜃23,𝜃24 = np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1))
𝜃31,𝜃32,𝜃33,𝜃34 = np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1))

𝛼1,𝛽1,𝛾1,𝛿1 =  np.random.uniform(-1, 1, (l1_neurons, 1)) ,np.random.uniform(-1, 1, (l1_neurons, 1)),np.random.uniform(-1, 1, (l1_neurons, 1)),np.random.uniform(-1, 1, (l1_neurons, 1))
𝛼2,𝛽2,𝛾2,𝛿2 = np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1))
𝛼3,𝛽3,𝛾3,𝛿3 = np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1))

# Regularization parameters
lambda_reg =0.0001

# Training parameters
learning_rate = 0.02
epochs = 80  # Train sample by sample

𝜃11_train,𝜃12_train,𝜃13_train,𝜃14_train = np.zeros((epochs,l1_neurons)),np.zeros((epochs,l1_neurons)),np.zeros((epochs,l1_neurons)),np.zeros((epochs,l1_neurons))
𝜃21_train,𝜃22_train,𝜃23_train,𝜃24_train = np.zeros((epochs,l2_neurons)),np.zeros((epochs,l2_neurons)),np.zeros((epochs,l2_neurons)),np.zeros((epochs,l2_neurons))
𝜃31_train,𝜃32_train,𝜃33_train,𝜃34_train = np.zeros((epochs,output_neurons)),np.zeros((epochs,output_neurons)),np.zeros((epochs,output_neurons)),np.zeros((epochs,output_neurons))

𝛼1_train,𝛽1_train,𝛾1_train,𝛿1_train = np.zeros((epochs,l1_neurons)),np.zeros((epochs,l1_neurons)),np.zeros((epochs,l1_neurons)),np.zeros((epochs,l1_neurons))
𝛼2_train,𝛽2_train,𝛾2_train,𝛿2_train = np.zeros((epochs,l2_neurons)),np.zeros((epochs,l2_neurons)),np.zeros((epochs,l2_neurons)),np.zeros((epochs,l2_neurons))
𝛼3_train,𝛽3_train,𝛾3_train,𝛿3_train = np.zeros((epochs,output_neurons)),np.zeros((epochs,output_neurons)),np.zeros((epochs,output_neurons)),np.zeros((epochs,output_neurons))

mse_train = np.zeros((epochs,num_class))
mse_test = np.zeros((epochs,num_class)) 

In [ ]:
def 𝑙𝑜𝑔𝑠𝑖𝑔(x,𝜃): 
    return 1/(1 + (scipy.special.expit(-1 * x * 𝜃.T)))

def 𝑡𝑎𝑛𝑠𝑖𝑔(x,𝜃):
    return 2/(1 + scipy.special.expit(-2 * x * 𝜃.T))-1

def relu(x,𝜃):
    return np.maximum(0, x * 𝜃.T)

def 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(x,𝜃):
    return x * 𝜃.T


# 𝐹1(𝑥) = 𝛼 × 𝑙𝑜𝑔𝑠𝑖𝑔(𝜃1,𝑥) + 𝛽 × 𝑡𝑎𝑛𝑠𝑖𝑔(𝜃2,𝑥) + 𝛾 × 𝑅𝑒𝑙𝑢(𝜃3,𝑥) 
def Function2(x,𝜃1,𝜃2,𝜃3,𝛼,𝛽,𝛾): 
    return 𝛼.T * 𝑙𝑜𝑔𝑠𝑖𝑔(x,𝜃1) + 𝛽.T * 𝑡𝑎𝑛𝑠𝑖𝑔(x,𝜃2) + 𝛾.T * relu(x,𝜃3)

def 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_x(x,𝜃):
#     return 𝜃.T * sigmoid_derivative(x,𝜃) * scipy.special.expit(-𝜃.T * x) / (1 + scipy.special.expit(-𝜃.T * x))**2
    return scipy.special.expit(-1 * x * 𝜃.T) * 𝜃.T * (1 - scipy.special.expit(-1 * x * 𝜃.T))

def 𝑡𝑎𝑛𝑠𝑖𝑔_derivative_x(x,𝜃):
#     return 𝜃.T * 2 * (1 + 𝑡𝑎𝑛𝑠𝑖𝑔(x,𝜃)) * (1 - 𝑡𝑎𝑛𝑠𝑖𝑔(x,𝜃)) * scipy.special.expit(-𝜃.T * 2 * x) / (1 + scipy.special.expit(-𝜃.T * 2 * x))**2
    return 4 * 𝜃.T * scipy.special.expit(-2 * x * 𝜃.T) / (1 + scipy.special.expit(-2 * x * 𝜃.T))**2

def relu_derivative_x(x,𝜃):
    return  𝜃.T * np.where(x * 𝜃.T > 0, 1, 0)

def 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_derivative_x(x,𝜃):
    return 𝜃.T


def Function2_derivative_x(x, 𝜃1, 𝜃2, 𝜃3, 𝛼, 𝛽, 𝛾): 
    result = 𝛼 * 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_x(x,𝜃1).T + 𝛽 * 𝑡𝑎𝑛𝑠𝑖𝑔_derivative_x(x,𝜃2).T + 𝛾 * relu_derivative_x(x,𝜃3).T
    return np.diag(result.flatten())


def 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_𝜃(x,𝜃):
#     return x * sigmoid_derivative_(x,𝜃) * scipy.special.expit(-𝜃.T * x) / (1 + scipy.special.expit(-𝜃.T * x))**2
    return scipy.special.expit(-1 * x * 𝜃.T) * x * (1 - scipy.special.expit(-1 * x * 𝜃.T))

def 𝑡𝑎𝑛𝑠𝑖𝑔_derivative_𝜃(x,𝜃):
#     return x * 2 * (1 + 𝑡𝑎𝑛𝑠𝑖𝑔(x,𝜃)) * (1 - 𝑡𝑎𝑛𝑠𝑖𝑔(x,𝜃)) * scipy.special.expit(-𝜃.T * 2 * x) / (1 + scipy.special.expit(-𝜃.T * 2 * x))**2
    return 4 * x * scipy.special.expit(-2 * x * 𝜃.T) / (1 + scipy.special.expit(-2 * x * 𝜃.T))**2

def relu_derivative_𝜃(x,𝜃):
    return x * np.where(x * 𝜃.T > 0,1, 0)

def 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_derivative_𝜃(x,𝜃):
    return x  

def Function1_derivative_𝜃(x, 𝛼, 𝛽, 𝛾, 𝛿, 𝜃1, 𝜃2, 𝜃3, 𝜃4): 
    result = 𝛼 * 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_𝜃(x,𝜃1).T + 𝛽 * 𝑡𝑎𝑛𝑠𝑖𝑔_derivative_𝜃(x,𝜃2).T + 𝛾 * relu_derivative_𝜃(x,𝜃3).T + 𝛿 * 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_derivative_𝜃(x,𝜃4).T
    return np.diag(result.flatten())

def Function2_derivative_𝜃(x, 𝛼, 𝛽, 𝛾, 𝜃1, 𝜃2, 𝜃3): 
    result = 𝛼 * 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_𝜃(x,𝜃1).T + 𝛽 * 𝑡𝑎𝑛𝑠𝑖𝑔_derivative_𝜃(x,𝜃2).T + 𝛾 * relu_derivative_𝜃(x,𝜃3).T 


In [ ]:
# Training the MLP for regression
for epoch in range(epochs):
    print('Epoch: {} \t'.format(epoch+1)) 
    # Shuffle the training data in each epoch
#     shuffle_indices = np.arange(len(X_train))
#     np.random.shuffle(shuffle_indices)
#     X_train = X_train[shuffle_indices]
#     y_train = y_train[shuffle_indices]
 
    error_data_train = np.zeros((len(X_train),num_class))
    output_data_train = np.zeros(len(X_train)) 
    for i in range(len(X_train)):
        #-------------------------------- Feed Forward -------------------------------------
        input_layer = X_train[i:i+1]

        net1 = np.dot(input_layer, w1)   # net1 = x * w1
        o1   = Function2(net1, 𝜃11, 𝜃12, 𝜃13,𝛼1,𝛽1,𝛾1) #  o1 = f(net1)
        net2 = np.dot(o1, w2) # net2 = o1 * w2
        o2   = Function2(net2, 𝜃21, 𝜃22, 𝜃23,𝛼2,𝛽2,𝛾2) #  o2 = f(net2)
        net3 = np.dot(o2, w3)   # net3 = o2 * w3
        o3   = Function2(net3, 𝜃31, 𝜃32, 𝜃33,𝛼3,𝛽3,𝛾3) #  o3 = f(net3)
        
        output_data_train[i] = np.argmax(o3)+1
        
        #-------------------------------- Backpropagation ----------------------------------- 
        output_layer_error = ((y_one_hot_train[i:i+1]).flatten() - o3.flatten()).reshape(1,-1) 

        # dE/da = dE/de * de/do3 * do3/da => o3=F(net3,a,𝜃3) = a*logsig + ... => dE/da = e * -1 * logsig(net3,𝜃3)
        diag_𝑙𝑜𝑔𝑠𝑖𝑔_net3 = np.array(np.diag(𝑙𝑜𝑔𝑠𝑖𝑔(net3.flatten(),𝜃31.flatten())))
        diag_𝑡𝑎𝑛𝑠𝑖𝑔_net3 = np.array(np.diag(𝑡𝑎𝑛𝑠𝑖𝑔(net3.flatten(),𝜃32.flatten())))
        diag_relu_net3 = np.array(np.diag(relu(net3.flatten(),𝜃33.flatten()))) 
        diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net3 = np.array(np.diag(𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(net3.flatten(),𝜃34.flatten())))
        
        d𝛼3 = -1 * (output_layer_error @ diag_𝑙𝑜𝑔𝑠𝑖𝑔_net3).T
        v𝛼3 = beta * v𝛼3 + (1 - beta) * d𝛼3
        𝛼3 = 𝛼3 - learning_rate * v𝛼3 - lambda_reg * 𝛼3
#         print('𝛼3: ',𝛼3)
        
        d𝛽3 = -1 * (output_layer_error @ diag_𝑡𝑎𝑛𝑠𝑖𝑔_net3).T
        v𝛽3 = beta * v𝛽3 + (1 - beta) * d𝛽3
        𝛽3 = 𝛽3 - learning_rate * v𝛽3 - lambda_reg * 𝛽3
#         print('𝛽3: ',𝛽3)
        
        d𝛾3 = -1 * (output_layer_error @ diag_relu_net3).T
        v𝛾3 = beta * v𝛾3 + (1 - beta) * d𝛾3
        𝛾3 = 𝛾3 - learning_rate * v𝛾3 - lambda_reg * 𝛾3
#         print('𝛾3: ',𝛾3)
        
        d𝛿3 = -1 * (output_layer_error @ diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net3).T
        v𝛿3 = beta * v𝛿3 + (1 - beta) * d𝛿3
        𝛿3 = 𝛿3 - learning_rate * v𝛿3 - lambda_reg * 𝛿3
#         print('𝛿3: ',𝛿3)
        
        # update 𝜃31: dE/de*de/do3*do3/𝜃31 = e * -1 * fprim_𝜃31 => fprim_𝜃31 = 𝛼3 * 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_𝜃
        diag_fprim_𝜃31 = np.array(np.diag((𝛼3.T * 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_𝜃(net3,𝜃31)).flatten()))
        d𝜃31 = -1 * (output_layer_error @ diag_fprim_𝜃31).T
        v𝜃31 = beta * v𝜃31 + (1 - beta) * d𝜃31
        𝜃31 = 𝜃31 - learning_rate * v𝜃31 - lambda_reg * 𝜃31
#         print('𝜃31: ',𝜃31)
        
        diag_fprim_𝜃32 = np.array(np.diag((𝛽3.T * 𝑡𝑎𝑛𝑠𝑖𝑔_derivative_𝜃(net3,𝜃32)).flatten()))
        d𝜃32 = -1 *  (output_layer_error @ diag_fprim_𝜃32).T
        v𝜃32 = beta * v𝜃32 + (1 - beta) * d𝜃32
        𝜃32 = 𝜃32 - learning_rate * v𝜃32 - lambda_reg * 𝜃32
#         print('𝜃32: ',𝜃32)
        
        diag_fprim_𝜃33 = np.array(np.diag((𝛾3.T * relu_derivative_𝜃(net3,𝜃33)).flatten()))
        d𝜃33 = -1 *  (output_layer_error @ diag_fprim_𝜃33).T
        v𝜃33 = beta * v𝜃33 + (1 - beta) * d𝜃33
        𝜃33 = 𝜃33 - learning_rate * v𝜃33 - lambda_reg * 𝜃33
#         print('𝜃33: ',𝜃33)
        
        diag_fprim_𝜃34 = np.array(np.diag((𝛿3.T * 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_derivative_𝜃(net3,𝜃34)).flatten()))
        d𝜃34 = -1 * (output_layer_error @ diag_fprim_𝜃34).T
        v𝜃34 = beta * v𝜃34 + (1 - beta) * d𝜃34
        𝜃34 = 𝜃34 - learning_rate * v𝜃34 - lambda_reg * 𝜃34
#         print('𝜃34: ',𝜃34)
        
        # update w3 with momentum
        w3_old = w3
        #Function1_derivative_x(x, 𝜃1, 𝜃2, 𝜃3, 𝜃4, 𝛼, 𝛽, 𝛾, 𝛿)
#         fprim_net3 = np.array(Function1_derivative_x(net3, 𝜃31, 𝜃32, 𝜃33, 𝜃34,𝛼3,𝛽3,𝛾3,𝛿3))
#         Function2_derivative_x(x, 𝜃1, 𝜃2, 𝜃3, 𝛼, 𝛽, 𝛾)
        fprim_net3 = np.array(Function2_derivative_x(net3, 𝜃31, 𝜃32, 𝜃33,𝛼3,𝛽3,𝛾3))
        dw3 = -1 * o2.T @ output_layer_error @ fprim_net3  
        vw3 = beta * vw3 + (1 - beta) * dw3 
        w3 = w3 - learning_rate * vw3 - lambda_reg * w3
#         print('w3: ',w3)
        
        diag_𝑙𝑜𝑔𝑠𝑖𝑔_net2 = np.array(np.diag(𝑙𝑜𝑔𝑠𝑖𝑔(net2,𝜃21).flatten())) 
        diag_𝑡𝑎𝑛𝑠𝑖𝑔_net2= np.array(np.diag(𝑡𝑎𝑛𝑠𝑖𝑔(net2,𝜃22).flatten()))  
        diag_relu_net2 = np.array(np.diag(relu(net2,𝜃23).flatten())) 
        diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net2 = np.array(np.diag(𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(net2,𝜃24).flatten()))  
        
        # dE/da = dE/de * de/do3 * do3/dnet3 * dnet3/do2 * do2/da2 
        # => o2=F(net2,a)= a*logsig + ... => dE/da = e * -1 *F'(net3) * w3_old * logsig(net2)
        d𝛼2 = -1 * diag_𝑙𝑜𝑔𝑠𝑖𝑔_net2 @ w3_old @ fprim_net3 @ output_layer_error.T 
        v𝛼2 = beta * v𝛼2 + (1 - beta) * d𝛼2
        𝛼2 = 𝛼2 - learning_rate * v𝛼2 - lambda_reg * 𝛼2
#         print('𝛼2: ',𝛼2.shape)
        
        d𝛽2 = -1 * diag_𝑡𝑎𝑛𝑠𝑖𝑔_net2 @  w3_old @ fprim_net3 @ output_layer_error.T
        v𝛽2 = beta * v𝛽2 + (1 - beta) * d𝛽2
        𝛽2 = 𝛽2 - learning_rate * v𝛽2 - lambda_reg * 𝛽2
#         print('𝛽2: ',𝛽2.shape)
        
        d𝛾2 = -1 * diag_relu_net2 @ w3_old @ fprim_net3 @ output_layer_error.T
        v𝛾2 = beta * v𝛾2 + (1 - beta) * d𝛾2
        𝛾2 = 𝛾2 - learning_rate * v𝛾2 - lambda_reg * 𝛾2
#         print('𝛾2: ',𝛾2.shape)
        
        d𝛿2 = -1 * diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net2 @ w3_old @ fprim_net3 @ output_layer_error.T
        v𝛿2 = beta * v𝛿2 + (1 - beta) * d𝛿2
        𝛿2 = 𝛿2 - learning_rate * v𝛿2 - lambda_reg * 𝛿2
    
        # update 𝜃21: dE/de*de/do3*do3/net3*dnet3/do2*do2/d𝜃21 = e * -1 * fprim_net3 * w3 * fprim_𝜃21 => fprim_𝜃21 = 𝛼2 * 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_𝜃
        diag_fprim_𝜃21 = np.array(np.diag((𝛼2.T * 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_𝜃(net2,𝜃21)).flatten()))
        d𝜃21 = -1 * diag_fprim_𝜃21 @ w3_old @ fprim_net3 @ output_layer_error.T
        v𝜃21 = beta * v𝜃21 + (1 - beta) * d𝜃21
        𝜃21 = 𝜃21 - learning_rate * v𝜃21 - lambda_reg * 𝜃21
        
        diag_fprim_𝜃22 = np.array(np.diag((𝛽2.T * 𝑡𝑎𝑛𝑠𝑖𝑔_derivative_𝜃(net2,𝜃22)).flatten()))
        d𝜃22 = -1 * diag_fprim_𝜃22 @ w3_old @ fprim_net3 @ output_layer_error.T
        v𝜃22 = beta * v𝜃22 + (1 - beta) * d𝜃22
        𝜃22 = 𝜃22 - learning_rate * v𝜃22 - lambda_reg * 𝜃22
        
        diag_fprim_𝜃23 = np.array(np.diag((𝛾2.T * relu_derivative_𝜃(net2,𝜃23)).flatten()))
        d𝜃23 = -1 * diag_fprim_𝜃23 @ w3_old @ fprim_net3 @ output_layer_error.T
        v𝜃23 = beta * v𝜃23 + (1 - beta) * d𝜃23
        𝜃23 = 𝜃23 - learning_rate * v𝜃23 - lambda_reg * 𝜃23 
        
        diag_fprim_𝜃24 = np.array(np.diag((𝛿2.T * 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_derivative_𝜃(net2,𝜃24)).flatten())) 
        d𝜃24 = -1 * diag_fprim_𝜃24 @ w3_old @ fprim_net3 @ output_layer_error.T
        v𝜃24 = beta * v𝜃24 + (1 - beta) * d𝜃24 
        𝜃24 = 𝜃24 - learning_rate * v𝜃24 - lambda_reg * 𝜃24
    
        # update w2 with momentum
        w2_old = w2
        # Create a diagonal matrix from the 1D array
#         fprim_net2 = np.array(Function1_derivative_x(net2, 𝜃21, 𝜃22, 𝜃23, 𝜃24,𝛼2,𝛽2,𝛾2,𝛿2))
        fprim_net2 = np.array(Function2_derivative_x(net3, 𝜃31, 𝜃32, 𝜃33,𝛼3,𝛽3,𝛾3))
#       w2 = w2 + learning_rate * output_layer_error * w3_old.dot(np.dot(np.dot(o1.T,fprim_net2),fprim_net3))

        #For Function1_derivative_x
#         dw2 = -1 * o1.T @ output_layer_error @ fprim_net3 @ w3_old.T @ fprim_net2

        #For Function2_derivative_x
        dw2 = -1 * o1.T @ output_layer_error @ fprim_net3 @ fprim_net2 @ w3_old.T 
        
        vw2 = beta * vw2 + (1 - beta) * dw2 
        w2 = w2 - learning_rate * vw2 - lambda_reg * w2
    
        # dE/da = dE/de * de/do3 * do3/dnet3 * dnet3/do2 * do2/dnet2 *dnet2/do1 * do1/da1
        # => o1=F(net1,a)= a*logsig + ... => dE/da = e * -1 *F'(net3) * w3 * F'(net2) *w2 * logsig(net1)
        diag_𝑙𝑜𝑔𝑠𝑖𝑔_net1 = np.array(np.diag(𝑙𝑜𝑔𝑠𝑖𝑔(net1,𝜃11).flatten()))
        diag_𝑡𝑎𝑛𝑠𝑖𝑔_net1= np.array(np.diag(𝑡𝑎𝑛𝑠𝑖𝑔(net1,𝜃12).flatten()))
        diag_relu_net1 = np.array(np.diag(relu(net1,𝜃13).flatten()))
        diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net1= np.array(np.diag(𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(net1,𝜃14).flatten()))
        
         #For Function1_derivative_x
#         d𝛼1 = -1 * diag_𝑙𝑜𝑔𝑠𝑖𝑔_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T 
        
        #For Function2_derivative_x
        d𝛼1 = -1 * diag_𝑙𝑜𝑔𝑠𝑖𝑔_net1 @ w2_old @ w3_old @ fprim_net2  @ fprim_net3 @ output_layer_error.T
        v𝛼1 = beta * v𝛼1 + (1 - beta) * d𝛼1
        𝛼1 = 𝛼1 - learning_rate * v𝛼1 - lambda_reg * 𝛼1
        
        #For Function1_derivative_x
#         d𝛽1 = -1 * diag_𝑡𝑎𝑛𝑠𝑖𝑔_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T 
        
        #For Function2_derivative_x
        d𝛽1 = -1 * diag_𝑡𝑎𝑛𝑠𝑖𝑔_net1 @ w2_old @ w3_old @ fprim_net2  @ fprim_net3 @ output_layer_error.T
        v𝛽1 = beta * v𝛽1 + (1 - beta) * d𝛽1
        𝛽1 = 𝛽1 - learning_rate * v𝛽1 - lambda_reg * 𝛽1
        
        #For Function1_derivative_x
#         d𝛾1 = -1 * diag_relu_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T 
        
        #For Function2_derivative_x
        d𝛾1 = -1 * diag_relu_net1 @ w2_old @ w3_old @ fprim_net2  @ fprim_net3 @ output_layer_error.T 
        v𝛾1 = beta * v𝛾1 + (1 - beta) * d𝛾1
        𝛾1 = 𝛾1 - learning_rate * v𝛾1 - lambda_reg * 𝛾1
        
        #For Function1_derivative_x
#         d𝛿1 = -1 * diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T 
        
        #For Function2_derivative_x
        d𝛿1 = -1 * diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net1 @ w2_old @ w3_old @ fprim_net2 @ fprim_net3 @ output_layer_error.T 
        v𝛿1 = beta * v𝛿1 + (1 - beta) * d𝛿1
        𝛿1 = 𝛿1 - learning_rate * v𝛿1 - lambda_reg * 𝛿1
        
        # update 𝜃11: dE/de*de/do3*do3/net3*dnet3/do2*do2/dnet2*dnet2/do1*do1/d𝜃11 = e * -1 * fprim_net3 * w3 * fprim_net2 * w2 * fprim_𝜃11 => fprim_𝜃11 = 𝛼1 * 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_𝜃
        diag_fprim_𝜃11 = np.array(np.diag((𝛼1.T * 𝑙𝑜𝑔𝑠𝑖𝑔_derivative_𝜃(net1,𝜃11)).flatten()))
        
        #For Function1_derivative_x
#         d𝜃11 = -1 * diag_fprim_𝜃11 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3  @ output_layer_error.T

        #For Function2_derivative_x
        d𝜃11 = -1 * diag_fprim_𝜃11 @ w2_old @ w3_old @ fprim_net2  @ fprim_net3  @ output_layer_error.T
        v𝜃11 = beta * v𝜃11 + (1 - beta) * d𝜃11
        𝜃11 = 𝜃11 - learning_rate * v𝜃11 - lambda_reg * 𝜃11
        
        diag_fprim_𝜃12 = np.array(np.diag((𝛽1.T * 𝑡𝑎𝑛𝑠𝑖𝑔_derivative_𝜃(net1,𝜃12)).flatten()))
        
        #For Function1_derivative_x
#         d𝜃12 = -1 * diag_fprim_𝜃12 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T

        #For Function2_derivative_x
        d𝜃12 = -1 * diag_fprim_𝜃12 @ w2_old @ w3_old @ fprim_net2  @ fprim_net3 @ output_layer_error.T
        v𝜃12 = beta * v𝜃12 + (1 - beta) * d𝜃12
        𝜃12 = 𝜃12 - learning_rate * v𝜃12 - lambda_reg * 𝜃12
        
        diag_fprim_𝜃13 = np.array(np.diag((𝛾1.T * relu_derivative_𝜃(net1,𝜃13)).flatten()))
        #For Function1_derivative_x
#         d𝜃13 = -1 *  diag_fprim_𝜃13 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T
        #For Function2_derivative_x
        d𝜃13 = -1 *  diag_fprim_𝜃13 @ w2_old @ w3_old @ fprim_net2 @ fprim_net3 @ output_layer_error.T
        v𝜃13 = beta * v𝜃13 + (1 - beta) * d𝜃13
        𝜃13 = 𝜃13 - learning_rate * v𝜃13 - lambda_reg * 𝜃13
         
        diag_fprim_𝜃14 = np.array(np.diag((𝛿1.T * 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_derivative_𝜃(net1,𝜃14)).flatten()))
        
        #For Function1_derivative_x
#         d𝜃14 = -1 *  diag_fprim_𝜃14 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T
        #For Function1_derivative_x
        d𝜃14 = -1 *  diag_fprim_𝜃14 @ w2_old @ w3_old @ fprim_net2 @ fprim_net3 @ output_layer_error.T
        v𝜃14 = beta * v𝜃14 + (1 - beta) * d𝜃14
        𝜃14 = 𝜃14 - learning_rate * v𝜃14 - lambda_reg * 𝜃14
        
        # update w1 with momentum      
#         fprim_net1 = np.array(Function1_derivative_x(net1, 𝜃11, 𝜃12, 𝜃13, 𝜃14,𝛼1,𝛽1,𝛾1,𝛿1))
        fprim_net1 = np.array(Function2_derivative_x(net1, 𝜃11, 𝜃12, 𝜃13,𝛼1,𝛽1,𝛾1))
        
        #For Function1_derivative_x
#         dw1 = -1 * input_layer.T @ output_layer_error @ fprim_net3 @ w3_old.T @ fprim_net2 @ w2_old.T @ fprim_net1
        #For Function2_derivative_x
        dw1 = -1 * input_layer.T @ output_layer_error @ fprim_net3 @ fprim_net2 @ w3_old.T @ w2_old.T @ fprim_net1
        vw1 = beta * vw1 + (1 - beta) * dw1 
        w1 = w1 - learning_rate * vw1 - lambda_reg * w1

        error_data_train[i] = output_layer_error 
    
    mse_train[epoch] = np.mean(error_data_train ** 2, axis=0)
    
    𝜃11_train[epoch] = 𝜃11.T
    𝜃12_train[epoch] = 𝜃12.T
    𝜃13_train[epoch] = 𝜃13.T
    𝜃14_train[epoch] = 𝜃14.T
    
    𝜃21_train[epoch] = 𝜃21.T
    𝜃22_train[epoch] = 𝜃22.T
    𝜃23_train[epoch] = 𝜃23.T
    𝜃24_train[epoch] = 𝜃24.T
    
    𝜃31_train[epoch] = 𝜃31.T
    𝜃32_train[epoch] = 𝜃32.T
    𝜃33_train[epoch] = 𝜃33.T
    𝜃34_train[epoch] = 𝜃34.T
    
    𝛼1_train[epoch] = 𝛼1.T
    𝛽1_train[epoch] = 𝛽1.T
    𝛾1_train[epoch] = 𝛾1.T
    𝛿1_train[epoch] = 𝛿1.T
    
    𝛼2_train[epoch] = 𝛼2.T
    𝛽2_train[epoch] = 𝛽2.T
    𝛾2_train[epoch] = 𝛾2.T
    𝛿2_train[epoch] = 𝛿2.T
    
    𝛼3_train[epoch] = 𝛼3.T
    𝛽3_train[epoch] = 𝛽3.T
    𝛾3_train[epoch] = 𝛾3.T
    𝛿3_train[epoch] = 𝛿3.T
    
    
    plt.figure(1, figsize=(10, 8))

    # Plotting the first subplot
    plt.subplot(2, 2, 1)
    plt.plot(data[:num_train, num_col-1], '-sr', label='Target Data')
    plt.plot(output_data_train, '-*b',label='Predicted Data')
    plt.xlabel('Train Data')
    plt.ylabel('Output')
    plt.legend()
    
    # Plotting the second subplot
    plt.subplot(2, 2, 2)
#     plt.semilogy(np.arange(1, epoch + 1), mse_train[:epoch],label=['MSE1','MSE2','MSE3'])
#     plt.semilogy(np.arange(1, epoch + 1), mse_train[:epoch],label=['MSE1','MSE2'])
    # Create labels dynamically
    labels = [f'MSE{i}' for i in range(1, num_class + 1)]
    plt.semilogy(np.arange(1, epoch + 1), mse_train[:epoch], label=labels)

    plt.xlabel('Epoch')
    plt.ylabel('MSE Train')
    plt.legend()
    # Adjust layout for better spacing
    plt.tight_layout()

    # Show the plots
    plt.show()
    
    print('MSE_train:  ', mse_train[epoch])

    print("\n\033[1;m" + "*" * 125)
        

#     if epoch % 100 == 0:
#         print(f"Epoch {epoch}: Total Error = {total_error[epoch]}")

print("****************************** Training completed *******************************")


In [ ]:
# Testing the trained MLP 
total_accuracy = 0 
error_data_test = np.zeros((len(X_test),num_class))
output_data_test = np.zeros(len(X_test))
for i in range(len(X_test)):
    input_layer = X_test[i:i+1]

    net1 = np.dot(input_layer, w1)   # net1 = x * w1
    o1   = Function2(net1, 𝜃11, 𝜃12, 𝜃13, 𝛼1,𝛽1,𝛾1) #  o1 = f(net1)
      
    net2 = np.dot(o1, w2) # net2 = o1 * w2
    o2   = Function2(net2,𝜃21, 𝜃22, 𝜃23,𝛼2,𝛽2,𝛾2) #  o2 = f(net2)
        
    net3 = np.dot(o2, w3)   # net3 = o2 * w3
    o3   = Function2(net3,𝜃31, 𝜃32, 𝜃33,𝛼3,𝛽3,𝛾3) #  o3 = f(net3)
    
    output_data_test[i] = np.argmax(o3)+1 
    error_data_test[i] = (y_one_hot_test[i:i+1]).flatten() - o3.flatten()

mse_test[epoch] = np.mean(error_data_test ** 2, axis=0)
plt.subplot(2, 2, 3)
plt.plot(data[num_train:, num_col-1], '-sr')
plt.plot(output_data_test, '-*b')
plt.xlabel('Test Data')
plt.ylabel('Output')

print('Epoch: '.format(epoch+1))
print('MSE_train:   ',mse_train[epoch],'\t, ','MSE_Test:  ', mse_test[epoch])

plt.tight_layout()
plt.show()


mse_train_result = mse_train[-1]
mse_test_result = mse_test[-1]

print("Final MSE on Train Data:", mse_train_result)
print("Final MSE on Test Data:", mse_test_result)

train_accuracy = np.mean(output_data_train == y_train) 
print(f"Accuracy on the train set: {train_accuracy * 100}%")

test_accuracy = np.mean(output_data_test == y_test) 
print(f"Accuracy on the test set: {test_accuracy * 100}%")


In [ ]:

# Assuming you have converted regression predictions to classes
predicted_classes = output_data_train 
actual_classes = data[:num_train,num_col-1] 

# Define the number of classes

# Create a confusion matrix-like matrix
confusion_matrix = np.zeros((num_class, num_class))

# Fill the confusion matrix
for actual, predicted in zip(actual_classes, predicted_classes):
    confusion_matrix[actual.astype(int)-1][predicted.astype(int)-1] += 1

# Plot the confusion matrix
plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Train Confusion Matrix ')
plt.colorbar()

# Annotate the plot with numbers
for i in range(num_class):
    for j in range(num_class):
        plt.text(j, i, str(int(confusion_matrix[i, j])), fontsize=12, ha='center', va='center')  # Corrected

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(np.arange(num_class), np.arange(1, num_class + 1))
plt.yticks(np.arange(num_class), np.arange(1, num_class + 1))

plt.show()


In [ ]:

# Assuming you have converted regression predictions to classes
predicted_classes = output_data_test 
actual_classes = data[num_train:,num_col-1] 

# Define the number of classes

# Create a confusion matrix-like matrix
confusion_matrix = np.zeros((num_class, num_class))

# Fill the confusion matrix
for actual, predicted in zip(actual_classes, predicted_classes):
    confusion_matrix[actual.astype(int)-1][predicted.astype(int)-1] += 1

# Plot the confusion matrix
plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Test Confusion Matrix ')
plt.colorbar()

# Annotate the plot with numbers
for i in range(num_class):
    for j in range(num_class):
        plt.text(j, i, str(int(confusion_matrix[i, j])), fontsize=12, ha='center', va='center')  # Corrected
# Adjust ticks and labels to start from 1
        
plt.xlabel('Predicted')
plt.ylabel('Actual')
# plt.xticks(np.arange(num_class))
# plt.yticks(np.arange(num_class))
plt.xticks(np.arange(num_class), np.arange(1, num_class + 1))
plt.yticks(np.arange(num_class), np.arange(1, num_class + 1))

plt.show()



In [ ]:
from sklearn.linear_model import LinearRegression 

# Plotting regression 
plt.figure(figsize=(8, 6)) 
plt.scatter(y_train, output_data_train, color='blue', marker='o', label='Predicted vs. Actual') 

# Fitting linear regression line 
regression = LinearRegression() 
regression.fit(y_train.reshape(-1, 1), output_data_train.reshape(-1, 1))  # Reshape to 2D arrays

# Plotting Y=T line
plt.plot(y_train, y_train, color='black',  label='Y=T') 

# Plotting the fitted line (regression line) with dashed style
plt.plot(y_train, regression.predict(y_train.reshape(-1, 1)), color='red',  label='Fit') 

plt.title('Regression Train') 
plt.xlabel('Target Values') 
plt.ylabel('Predicted Outputs') 
plt.legend() 
plt.grid(True) 
plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression 

# Plotting regression 
plt.figure(figsize=(8, 6)) 
plt.scatter(y_test, output_data_test, color='blue', marker='o', label='Predicted vs. Actual') 

# Fitting linear regression line 
regression = LinearRegression() 
regression.fit(y_test.reshape(-1, 1), output_data_test.reshape(-1, 1))  # Reshape to 2D arrays

# Plotting Y=T line
plt.plot(y_test, y_test, color='black',  label='Y=T') 

# Plotting the fitted line (regression line) with dashed style
plt.plot(y_test, regression.predict(y_test.reshape(-1, 1)), color='red',  label='Fit') 

plt.title('Regression Test') 
plt.xlabel('Target Values') 
plt.ylabel('Predicted Outputs') 
plt.legend() 
plt.grid(True) 
plt.show()


In [ ]:
plt.figure(figsize=(15,30)) 

# Number of rows
num_rows = l2_neurons // 3 + (l2_neurons % 3 > 0)  # To ensure all neurons are covered

# Plotting 𝜃21_train for each neuron in the same row
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝜃21_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Theta1')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


print("\n\033[1;m" + "=" * 125)


plt.figure(figsize=(15,30)) 

# Plotting 𝜃22_train for each neuron in the same row
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝜃22_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Theta2')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


plt.figure(figsize=(15,30)) 

# Plotting 𝜃23_train for each neuron in the same row
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝜃23_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Theta3')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

print("\n\033[1;m" + "=" * 125)


plt.figure(figsize=(15,30)) 

# Plotting 𝜃24_train for each neuron in the same row
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝜃24_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Theta4')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
plt.figure(figsize=(15,5)) 

# Number of rows
num_rows = output_neurons // 3 + (output_neurons % 3 > 0)  # To ensure all neurons are covered

# Plotting 𝜃31_train for each neuron in the same row
for i in range(output_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝜃31_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Theta1')
    plt.title(f'Layer 3, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


print("\n\033[1;m" + "=" * 125)


plt.figure(figsize=(15,5)) 

# Plotting 𝜃32_train for each neuron in the same row
for i in range(output_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝜃32_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Theta2')
    plt.title(f'Layer 3, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


plt.figure(figsize=(15,5)) 

# Plotting 𝜃33_train for each neuron in the same row
for i in range(output_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝜃33_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Theta3')
    plt.title(f'Layer 3, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

print("\n\033[1;m" + "=" * 125)


plt.figure(figsize=(15,5)) 

# Plotting 𝜃24_train for each neuron in the same row
for i in range(output_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝜃34_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Theta4')
    plt.title(f'Layer 3, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
plt.figure(figsize=(15, 5)) 

# Plotting 𝛼1_train for each neuron in the same row
for i in range(l1_neurons):
    plt.subplot(1, l1_neurons, i+1)
    plt.plot(np.arange(0, epoch + 1), 𝛼1_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('alpha1')
    plt.title(f'Layer 1, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()
plt.show()


# Plotting 𝛽1_train for each neuron in the same row
plt.figure(figsize=(15, 5))
for i in range(l1_neurons):
    plt.subplot(1, l1_neurons, i+1)
    plt.plot(np.arange(0, epoch + 1), 𝛽1_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('beta1')
    plt.title(f'Layer 1, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()


# Plotting 𝛾1_train for each neuron in the same row
plt.figure(figsize=(15, 5))
for i in range(l1_neurons):
    plt.subplot(1, l1_neurons, i+1)
    plt.plot(np.arange(0, epoch + 1), 𝛾1_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('gama1')
    plt.title(f'Layer 1, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()


# Plotting 𝛿1_train for each neuron in the same row
plt.figure(figsize=(15, 5))
for i in range(l1_neurons):
    plt.subplot(1, l1_neurons, i+1)
    plt.plot(np.arange(0, epoch + 1), 𝛿1_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('delta1')
    plt.title(f'Layer 1, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()

In [ ]:
plt.figure(figsize=(15,30)) 

# Number of rows
num_rows = l2_neurons // 3 + (l2_neurons % 3 > 0)  # To ensure all neurons are covered

# Plotting 𝛼2_train for each neuron in the same row
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3) 
    plt.plot(np.arange(0, epoch + 1), 𝛼2_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('alpha2')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()
plt.show()


# Plotting 𝛽2_train for each neuron in the same row
plt.figure(figsize=(15, 30))
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛽2_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('beta2')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()


# Plotting 𝛾2_train for each neuron in the same row
plt.figure(figsize=(15, 30))
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛾2_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('gama2')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()


# Plotting 𝛿2_train for each neuron in the same row
plt.figure(figsize=(15, 30))
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛿2_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('delta2')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()

In [ ]:
plt.figure(figsize=(15,5)) 

# Number of rows
num_rows = output_neurons // 3 + (output_neurons % 3 > 0)  # To ensure all neurons are covered

# Plotting 𝛼3_train for each neuron in the same row
for i in range(output_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛼3_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('alpha3')
    plt.title(f'Layer 3, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


print("\n\033[1;m" + "=" * 125)


plt.figure(figsize=(15,5)) 

# Plotting 𝛽3_train for each neuron in the same row
for i in range(output_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛽3_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('beta3')
    plt.title(f'Layer 3, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


plt.figure(figsize=(15,5)) 

# Plotting 𝛾3_train for each neuron in the same row
for i in range(output_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛾3_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('gama3')
    plt.title(f'Layer 3, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

print("\n\033[1;m" + "=" * 125)


plt.figure(figsize=(15,5)) 

# Plotting 𝛿3_train for each neuron in the same row
for i in range(output_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛿3_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('delta4')
    plt.title(f'Layer 3, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()
